In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

In [2]:
es=Elasticsearch(hosts='http://localhost:9200/')
client_info=es.info()
print("Connected to Elastic-Search !")
pprint(client_info.body)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))))